# Domherren der Personendatenbank
## Datenimport / -update WIAGweb 1

Quellen: Personendatenbank (über https://mysql.gwdg.de/)  
C:\Users\georg\Documents\projekte-doc\gso\data

- [Personendaten](#Personendaten)
- [IDs zuordnen](#IDs-zuordnen)
- [Literaturangaben](#Literaturangaben)
- [Ämter](#Ämter)

2021-11-09, 2022-01-11, 2022-02-17

In [1]:
wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

"C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\Georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise

In [6]:
using WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1423


In [7]:
using MySQL

In [8]:
using DataFrames

In [9]:
Wds = WiagDataSetup

WiagDataSetup

In [10]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

Datenbank anlegen: `wiag_in`  
Tabellen anlegen: siehe `domherren_create.sql`
- `cn_canon_gs`
- `cn_office_gs`
- `cn_canon_reference_gs`
- `cn_reference_gs`

## Personendaten

Betrachte nur Personen, deren Eintrag nicht gelöscht ist und die online sind. Aus einem nicht erschließbaren Grund ist `item_id` nur eindeutig, wenn aus der Tabelle `gsn` nur der Eintrag mit der kleinsten ID berücksichtigt wird.
``` sql
select gsn.nummer, item_id, gsn.deleted, items.deleted, items.status from gsn join items on gsn.item_id = items.id where items.status = 'online' and not items.deleted limit 12 offset 4000;
select count(*) from gsn join items on gsn.item_id = items.id where items.status = 'online' and not items.deleted;
```
97014
``` sql
select count(distinct(gsn.nummer)) from gsn join items on gsn.item_id = items.id where items.status = 'online' and not items.deleted;
```
97013
``` sql
create temporary table item_online as
select min(gsn.id) as gsn_tbl_id, gsn.nummer, item_id, items.deleted, items.status
from gso_in_202201.gsn join gso_in_202201.items on gsn.item_id = items.id where items.status = 'online'
group by items.id;
```
87247

In [11]:
gs_db = "gso_in_202202"
sql = "SELECT min(gsn.id) as gsn_tbl_id, gsn.nummer as gsn_id, max(item_id) as item_id, items.status " * 
"FROM $(gs_db).gsn JOIN $(gs_db).items ON gsn.item_id = items.id " *
"WHERE items.status = 'online' AND NOT items.deleted " *
"GROUP BY item_id";
df_p_online = Wds.sql_df(sql);

In [12]:
size(df_p_online)

(80966, 4)

Ermittle die IDs der Domherren anhand der relevanten Stifte.

In [13]:
sql = "SELECT gs_id FROM wiag.domstift"
df_ds = Wds.sql_df(sql);

In [14]:
size(df_ds)

(34, 1)

In [15]:
str_ds = join(df_ds.gs_id, ", ")

"3498, 3492, 701, 343, 60130, 3493, 3496, 783, 675, 739, 832, 3503, 3499, 736, 676, 3488, 3501, 792, 794, 616, 628, 3491, 803, 953, 3495, 226, 3494, 679, 2066, 3489, 3500, 3487, 3490, 3502"

In [16]:
gs_db = "gso_in_202202"
sql = "SELECT DISTINCT person_id " *
"FROM $(gs_db).offices " *
"WHERE klosterid in (" * str_ds * ") "
df_ids_cn = Wds.sql_df(sql);

In [17]:
size(df_ids_cn)

(13265, 1)

In [18]:
str_ids_cn = join(df_ids_cn.person_id, ", ");

Lies Personendaten

In [19]:
sql = "SELECT p.id as id_person_gso, item_id, " *
"namenspraefix as prefix_name, vorname as givenname, familienname as familyname, " *
"sterbedatum as date_death, geburtsdatum as date_birth, `orden` as religious_order, " *
"anmerkungen as annotation_ed, titel as academic_title, " * 
"gndnummer as gnd_id, viaf as viaf_id, " *
"familiennamenvarianten as familyname_variant, vornamenvarianten as givenname_variant, " *
"namenszusatz as comment_name, " *
"zeitraum_von as date_hist_first, zeitraum_bis as date_hist_last " *
"FROM $(gs_db).persons as p " *
"WHERE p.id IN (" * str_ids_cn * ") " *
"ORDER BY p.id"
df_p_cn = Wds.sql_df(sql);

In [20]:
Wds.clean_up!(df_p_cn)

In [21]:
size(df_p_cn)

(13265, 17)

Ergänze die GSN und beschränke auf Domherren, die online sind.

In [22]:
df_p_cn = innerjoin(df_p_cn, df_p_online, on = :item_id);

In [23]:
size(df_p_cn)

(8564, 20)

In [24]:
names(df_p_cn)

20-element Vector{String}:
 "id_person_gso"
 "item_id"
 "prefix_name"
 "givenname"
 "familyname"
 "date_death"
 "date_birth"
 "religious_order"
 "annotation_ed"
 "academic_title"
 "gnd_id"
 "viaf_id"
 "familyname_variant"
 "givenname_variant"
 "comment_name"
 "date_hist_first"
 "date_hist_last"
 "gsn_tbl_id"
 "gsn_id"
 "status"

In [26]:
first(df_p_cn, 6)

,id_person_gso,item_id,prefix_name,givenname,familyname,date_death,date_birth
,Int32,Int32,SubStri…?,SubStri…?,SubStri…?,SubStri…?,SubStri…?
1,4586,4586,missing,Nikolaus,Amelung,missing,missing
2,4590,4590,von,Johann,Angern,1423,missing
3,4620,4620,missing,Lorenz,Bading,1557,missing
4,4623,4623,(von),Werner,Balck,missing,missing
5,4678,4678,(von),Johannes,Bellin,missing,missing
6,4684,4684,von,Werner,Berg,missing,missing


### IDs zuordnen
Lies bestehende IDs

In [24]:
sql = "SELECT id, id_person_gso FROM wiag.cn_canon_gs"
df_id_cn_wiag = Wds.sql_df(sql);

In [25]:
df_p_cn = leftjoin(df_p_cn, df_id_cn_wiag, on = :id_person_gso);

In [26]:
size(df_p_cn)

(8564, 21)

In [27]:
size(filter(ismissing, df_p_cn.id))

(191,)

Wer fehlt?

In [28]:
df_cn_gone = antijoin(df_id_cn_wiag, df_p_cn, on = :id_person_gso);

In [29]:
first(df_cn_gone, 5)

,id,id_person_gso
,Int32,Int32?
1,80838,20099
2,80971,23212
3,81163,47159
4,81175,55593
5,83499,142939


Das sind zum Beispiel bereinigte Dubletten (005-03705-001).  
**R** Domherren, die aus dem aktuellen Online-Bestand der Personendatenbank herausgenommen werden, werden über WIAG nicht mehr zu finden sein, weil der Redaktionsprozess der Personendatenbank in WIAG nicht nachgebildet wird.

In [116]:
df_p_cn[100:130, [:id, :id_person_gso, :gsn_tbl_id]];

Die Daten sind nach `gsn_tbl_id` sortiert (auch die anderen beiden IDs folgen dieser Sortierung)

In [30]:
df_p_cn_mg_id = filter(:id => ismissing, df_p_cn);

In [31]:
size(df_p_cn_mg_id)

(191, 21)

In [32]:
df_p_cn_id = filter(:id => !ismissing, df_p_cn);

In [33]:
maximum(df_p_cn_id.id_person_gso), size(df_p_cn_id)

(330858, (8373, 21))

In [113]:
minimum(df_p_cn_mg_id.id_person_gso), size(df_p_cn_mg_id)

(285187, (6, 21))

Sobald hier neue IDs vergeben werden, wird die Sortierung von ID (in WIAG) und `persons.id` nicht mehr kongruent sein.

In [37]:
id_max, s_cn_id = maximum(df_p_cn_id.id), size(df_p_cn_id)

(90041, (8373, 21))

In [34]:
id_person_gso = sort(df_p_cn_mg_id.id_person_gso);

In [35]:
last(df_id_cn_wiag, 8)

,id,id_person_gso
,Int32,Int32?
1,90034,330844
2,90035,330845
3,90036,330849
4,90037,330852
5,90038,330855
6,90039,330856
7,90040,330857
8,90041,330858


In [38]:
id_c = id_max + 1;
for id_gso in id_person_gso
    push!(df_id_cn_wiag, (id_c, id_gso))
    id_c += 1;
end

2021-11: `df_id_cn_wiag` hat jetzt 20 Zeilen mehr als `df_p_cn`: eben jene, die in der Quelle als Dubletten erkannt wurden oder aus anderen Gründen entfernt wurden.

In [39]:
length(unique(df_id_cn_wiag.id)), length(unique(df_id_cn_wiag.id_person_gso))

(8587, 8587)

Entferne die IDs wieder, die schon zugeordnet waren und füge sie wieder ein mit dem vervollständigten Satz von IDs.

In [40]:
df_p_cn = select(df_p_cn, Not(:id));

In [41]:
size(df_p_cn)

(8564, 20)

In [42]:
df_p_cn = leftjoin(df_p_cn, df_id_cn_wiag, on = :id_person_gso);

In [43]:
size(filter(:id => ismissing, df_p_cn))

(0, 21)

In [44]:
Wds.clean_up!(df_p_cn)

In [45]:
df_p_cn[20:30, [:id, :id_person_gso, :givenname, :date_death]]

,id,id_person_gso,givenname,date_death
,Int32,Int32,String?,String?
1,80020,4984,Engelbert,missing
2,80021,5049,Matthäus,missing
3,80022,5090,Joachim,1544
4,80023,5092,Jakob,missing
5,80024,5096,Peter,missing
6,80025,5135,Gebhard,missing
7,80026,5156,Heinrich,missing
8,80027,5166,Gerhard,missing
9,80028,5170,Nikolaus,missing


In [46]:
maximum(df_p_cn.id)

90232

In [47]:
numdate_birth(date) = Wds.parsemaybe(date, :lower)
df_p_cn[!, :numdate_birth] .= numdate_birth.(df_p_cn.date_birth);

In [48]:
numdate_death(date) = Wds.parsemaybe(date, :upper)
df_p_cn[!, :numdate_death] .= numdate_death.(df_p_cn.date_death);

In [49]:
table_name = "cn_canon_gs"
Wds.filltable!(table_name, select(df_p_cn, Not([:gsn_tbl_id, :item_id])), clear_table = true)

┌ Info: Rows inserted: 8564
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1201


8564

## Literaturangaben
Die in der Tabelle `locations` angegebenen Werte für `book_id` sind kongruent zu den IDs in `wiag.cn_reference_gs`.
``` sql
select count(*) from locations 
where book_id not in (select id from wiag.cn_reference_gs) 
and item_status = 'online' and not deleted;
```
0

Referenzwerke (`gso_in_202202.books`)
``` sql
source 'C:\Users\georg\Documents\projekte-doc\gso\data\books.sql';
```

In [50]:
gs_db

"gso_in_202202"

In [51]:
table_src = "$(gs_db).books"
sql = "SELECT id, titel AS title, autoren AS author, nummer AS number_vol, " *
"uri AS online_resource, kurztitel AS shorttitle " *
"FROM " * table_src 
df_vol = Wds.sql_df(sql);

In [52]:
size(df_vol)

(87, 6)

In [53]:
table_name = "cn_reference_gs"
Wds.filltable!(table_name, df_vol, clear_table = true)

┌ Info: Rows inserted: 87
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1201


87

Lies die Domherren ein mit `item_id`

In [54]:
sql = "SELECT c.id AS id_canon, p.item_id FROM cn_canon_gs AS c " *
"JOIN $(gs_db).persons as p on p.id = c.id_person_gso"
df_cn = Wds.sql_df(sql);

In [55]:
size(df_cn)

(8564, 2)

Lies die Literaturverweise ein

In [56]:
sql = "SELECT * FROM $(gs_db).locations " * 
"WHERE item_status = 'online' AND NOT deleted " *
"GROUP BY item_id, book_id, seiten"
df_p_ref = Wds.sql_df(sql);

In [57]:
size(df_p_ref)

(98321, 11)

Welche Domherren haben eine Kurz-Biographie?

In [58]:
is_bio(seiten) = contains(seiten, "<b>");
df_p_ref[!, :is_bio] .= is_bio.(df_p_ref.seiten);

In [59]:
sum(df_p_ref.is_bio)

35018

Übernimm für Domherren mit Kurzbiographie nur Literaturhinweise mit Kurzbiographien (`transfer_gso.sql`: 2021-04-26) Es ist freilich möglich, dass es mehrere Literaturverweise gibt, die eine Kurzbiographie enthalten.

In [60]:
df_p_ref_bio = filter(:is_bio => identity, df_p_ref);

In [61]:
size(df_p_ref_bio)

(35018, 12)

In [62]:
df_cn_ref_bio = innerjoin(df_cn, df_p_ref_bio, on = :item_id);

In [63]:
size(df_cn_ref_bio), length(unique(df_cn_ref_bio.id_canon))

((7420, 13), 6598)

In [64]:
df_cn_mg_bio = antijoin(df_cn, select(df_cn_ref_bio, :id_canon), on = :id_canon);

In [65]:
size(df_cn_mg_bio)

(1966, 2)

In [66]:
df_cn_ref_mg_bio = innerjoin(df_cn_mg_bio, df_p_ref, on = :item_id);

In [67]:
size(df_cn_ref_mg_bio)

(2434, 13)

Wandle `is_bio` in Int um.

In [68]:
transform!(df_cn_ref_bio, :is_bio => ByRow(Int) => :is_bio);

In [69]:
transform!(df_cn_ref_mg_bio, :is_bio => ByRow(Int) => :is_bio);

In [70]:
columns = [
    :id_canon => :id_canon,
    :book_id => :id_reference,
    :seiten => :page_reference,
    :is_bio => :isbio,
    :item_status => :status,
];

Schreibe die Einträge mit Kurzbiographie

In [71]:
table_name = "cn_canon_reference_gs"
Wds.filltable!(table_name, select(df_cn_ref_bio, columns), clear_table = true)

┌ Info: Rows inserted: 7420
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1201


7420

*2021-19-11* Einträge ohne Kurzbiographie werden nicht mehr ausgegeben; siehe Vorgängerversion der Daten. Diesen Schritt überspringen.
Schreibe die Einträge ohne Kurzbiographie

In [47]:
table_name = "cn_canon_reference_gs"
Wds.filltable!(table_name, select(df_cn_ref_mg_bio, columns), clear_table = false)

┌ Info: Rows inserted: 2418
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1184


2418

## Ämter

In [14]:
gs_db = "gso_in_202202"

"gso_in_202202"

In [15]:
sql = "SELECT person_id as id_person_gso, dioezese as diocese, bezeichnung as office_name, " * 
"von as date_start, bis as date_end, institution, ort as location, " *
"anmerkung as comment, klosterid as id_monastery " *
"FROM $(gs_db).offices " *
"WHERE NOT deleted"
df_oc = Wds.sql_df(sql);

In [16]:
Wds.clean_up!(df_oc)

In [17]:
table_name = "cn_canon_gs"
sql = "SELECT id AS id_canon, id_person_gso 
FROM $(table_name)";
df_p_id = Wds.sql_df(sql);

In [18]:
Wds.clean_up!(df_p_id)

In [19]:
size(df_oc)

(72637, 9)

In [21]:
df_o_cn = innerjoin(df_oc, df_p_id, on = :id_person_gso);

In [22]:
size(df_o_cn)

(14991, 10)

`location_show` und `location` stimmen für für die meisten Ämter der Domherren aus der Personendatenbank überein. `location_show` kann nur mit einer aktuellen Version von `cn_online` gesetzt werden.

In [77]:
df_o_cn[!, :location_show] .= df_o_cn.location;

In [23]:
date_start(date) = Wds.parsemaybe(date, :lower)
df_o_cn[!, :numdate_start] .= date_start.(df_o_cn.date_start);

In [24]:
date_end(date) = Wds.parsemaybe(date, :upper)
df_o_cn[!, :numdate_end] .= date_end.(df_o_cn.date_end);

Befülle `time_sortkey` erst aus `date_end`, dann aus `date_start`

In [61]:
df_o_cn[!, :time_sortkey] .= missing;

In [62]:
transform!(df_o_cn, :date_end => ByRow(Wds.parse_year_sort) => :time_sortkey);

In [70]:
function update_col(from, to)
    cand = Wds.parse_year_sort(from)
    if ismissing(to) || cand < to
        return cand
    else
        return to
    end
end

update_col (generic function with 1 method)

In [71]:
transform!(df_o_cn, [:date_start, :time_sortkey] => ByRow(update_col) => :time_sortkey);

In [40]:
ENV["COLUMNS"] = 150

150

In [47]:
df_o_cn[20:30, [:id_canon, :office_name, :date_start, :date_end, :numdate_start, :numdate_end, :time_sortkey]]

,id_canon,office_name,date_start,date_end,numdate_start,numdate_end,time_sortkey
,Int32,String?,String?,String?,Int64?,Int64?,Int64
1,84434,Domherr,1807,missing,1807,missing,1807150
2,84435,Domherr,1792,missing,1792,missing,1792150
3,81140,Propst,1559,1569/1571,1559,1569,1559150
4,81055,Dompropst,um 1150,missing,1145,missing,1150210
5,83575,Domherr,1873,1887,1873,1887,1873150
6,81378,Domherr,1356,missing,1356,missing,1356150
7,81407,Domherr,1732,missing,1732,missing,1732150
8,81423,Domherr,1396,missing,1396,missing,1396150
9,81439,Domherr,Ende 18. Jahrhundert,missing,1767,missing,1799594


Welche Bistümer sind in der Datenbank?

In [42]:
sql = "SELECT diocese FROM wiag.diocese"
df_dioc = Wds.sql_df(sql);

In [59]:
in_db(diocese) = ismissing(diocese) ? 0 : (in(diocese, df_dioc.diocese) ? 1 : 0)
df_o_cn[!, :diocese_in_db] .= 0;
for row in eachrow(df_o_cn)
    row[:diocese_in_db] = in_db(row[:diocese])
end

Das Domstift Krakau ist nicht in der Klosterdatenbank. Dafür sind aber die Felder `institution` und `location` befüllt.

In [72]:
table_name = "cn_office_gs"
Wds.filltable!(table_name, select(df_o_cn, Not(:id_person_gso)), clear_table = true)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1179
┌ Info: Rows inserted: 14991
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


14991

### `cn_office_gs.location_show`
`monastery_location` wird regelmäßig befüllt mit Daten aus der Klosterdatenbank.

In der Regel werden die Domherren aus der Domherrendatenbank später eingelesen, dann erst kann `cn_online` erstellt werden, und schließlich das Feld `cn_office_gs.location_show` befüllt werden.

In [84]:
Wds.fillofficelocation("cn_office_gs", "monastery_location", "wiag.place", "cn_online", "id_gs")

┌ Info: 1000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 3000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 5000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 7000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 8000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 9000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 10000
└ @ WiagDataSetup C:\Us

12284

Was ist mit der Diözese 'Hamburg-Bremen'?

Daten übernehmen
``` sql
use wiag;
delete from cn_canon_gs;
insert into cn_canon_gs (select * from wiag_in.cn_canon_gs);
```
anlog: `cn_canon_reference_gs`, `cn_office_gs`  
siehe auch Hilfstabellen in http://localhost:8888/notebooks/Documents/projekte/WiagDataSetup.jl/run/wiag/Domherren.ipynb